In [63]:
import numpy as np
import matplotlib.pyplot as plt

In [64]:
data = np.genfromtxt("cm_dataset_2.csv", delimiter=",")
n, d = data.shape
data = data - np.mean(data, axis=0)
max_iter = 100
k = 2
num_initializations = 1


# plt.scatter(data[:, 0], data[:, 1], c='black', label='Data Points')
# plt.title('Raw Data Scatter Points')
# plt.xlabel('X')
# plt.ylabel('Y')

In [65]:
def kernel_function(x, y) :
    return (np.dot(x,y) + 1) ** 2

# K = np.zeros((n,n))

# for i in range(n) :
#     for j in range(n) :
#         K[i,j] = kernel_function(data[i], data[j])

In [72]:
def kmeans_kernelized(data, k, max_iter=100, num_initializations=1):
    """Performs kernelized K-means clustering."""
    n, d = data.shape

    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = kernel_function(data[i], data[j])

    best_score = float("inf")
    best_labels = None

    for _ in range(num_initializations):
        # centroids = data.T[np.random.choice(n, k, replace=False)].T.reshape(k,-1)
        centroids = data.T[np.random.choice(n, k, replace=False), :].T
        for _ in range(max_iter):
            labels = np.argmax(centroids @ K.T, axis=1)
            for i in range(k):
                assigned_data = data[labels == i]
                if len(assigned_data) > 0:  
                    centroids[i] = K[labels == i, :].T @ assigned_data / np.sum(K[labels == i])
        score = np.sum(
            [np.linalg.norm(K[i, labels[i]] - K[i, centroids[j]]) ** 2 for i in range(n) for j in range(k) if labels[i] == j]
        )

        if score < best_score:
            best_score = score
            best_labels = labels

    return best_labels

In [73]:
labels = kmeans_kernelized(data, k)

# Visualize (optional)
plt.scatter(data[:, 0], data[:, 1], c=labels)
plt.show()

IndexError: index 229 is out of bounds for axis 0 with size 2

In [ ]:
for _ in range(num_initializations):
    centroids = data[np.random.choice(range(n), k, replace=False)]
    og_centroids = centroids
    max_iter = 100
    for _ in range(max_iter):
        distances = np.array([[np.dot(K[i] - K[j], K[i] - K[j]) for j in range(k)] for i in range(n)])
        labels = np.argmin(distances, axis=1)
        centroids = np.array([np.mean(data[labels == j], axis=0) for j in range(k)])

    colors = plt.cm.Spectral(np.linspace(0, 1, k))
    for cluster_label, color in zip(range(k), colors):
        cluster_points = data[labels == cluster_label]
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=[color], marker='o', label=f'Cluster {cluster_label + 1} - Initialization')

    plt.scatter(og_centroids[:, 0], og_centroids[:, 1], c='black', marker='o', label='init')
    plt.scatter(centroids[:, 0], centroids[:, 1], c='black', marker='*', s=150, label='mu')
# Plot styling
plt.title('Kernelized K-means Clustering with Different Initializations')
plt.xlabel('X')
plt.ylabel('Y')
# plt.legend()
plt.show()